[View in Colaboratory](https://colab.research.google.com/github/YejinYu/kaggle-dogbreed/blob/master/dog_breed_cnn.ipynb)

In [26]:
# 구글 콜랩에서 GPU 메모리가 너무 작게 배정되는 경우가 가끔씩 있어서 메모리 체크하려고 가져온 코드입니다. Preparation으로 바로 건너뛰시면 됩니다.
# 출처: https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available

# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 9.2 GB  | Proc size: 2.8 GB
GPU RAM Free: 548MB | Used: 10891MB | Util  95% | Total 11439MB


In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# # 2. Load a file by ID and print its contents.
file_id = '1nPYrpY_xzBLfnFrr1-mhdl1uYMoaP3D5'
drive_service = build('drive', 'v3')

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  progress, done = downloader.next_chunk()

with open('original_copies.zip', 'wb') as f:
  f.write(downloaded.getvalue())

In [0]:
!unzip original_copies.zip

In [7]:
%cd /content
!mkdir data
%cd data
!mkdir train test
%cd /content

/content
/content/data
/content


In [0]:
# train_test_split 를 정하고 random.random() 을 돌려 각 데이터를 /train 폴더나 /test 폴더에 복사한다

import os
from os.path import join
import random
import shutil

# For reproducibility
random.seed(42)

classes = ['dosamastiff', 'jindotgae', 'sapsalgae']
train_test_split = 0.2

for c in classes:
  dirpath = join('original_copies', c)
  files = os.listdir(dirpath)
  for f in files:
    fpath = join(dirpath, f)
    folder = 'test' if random.random() < train_test_split else 'train'
    new_dir = join('data', folder)
    shutil.copy(fpath, new_dir)

In [0]:
# 각 폴더(train/test)에 들어있는 데이터를 바탕으로 csv를 만든다.
import pandas as pd

for folder in ['train', 'test']:
    fnames = os.listdir(join('data',folder))  # train 혹은 test 폴더에 있는 모든 파일이름들

    ids = pd.Series(fnames)  # 이미지파일 이름 ex) 'jindotgae_104.jpg'
    labels = list(ids.map(lambda x: x.split('_')[0]))  # 종 ex) 'jindotgae'

    df = pd.DataFrame({'id':ids, 'breed':labels})
    df.to_csv(f'labels_{folder}.csv', index=False)

In [0]:
import numpy as np
import keras as k
import tensorflow as tf
import os

In [0]:
NUM_CLS = 3
SIZE = (299,299)
POOLING = 'avg'
BATCH_SIZE = 32
NUM_EPOCHS = 10

def img_and_label(img_id, train_test, size):
    cls = img_id.split('_')[0]
    img = image.load_img(os.path.join('data', train_test, img_id), target_size=size)
    img = image.img_to_array(img)
    return img, cls

In [0]:
labels_train = pd.read_csv('labels_train.csv')
X_train = np.zeros((len(labels_train), *SIZE, 3))
y_train = np.zeros((len(labels_train), NUM_CLS))

labels_test = pd.read_csv('labels_test.csv')
X_test = np.zeros((len(labels_test), *SIZE, 3))
y_test = np.zeros(len(labels_test))

y_template = {'dosamastiff':0, 'jindotgae':1, 'sapsalgae':2}

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(407, 299, 299, 3)
(407, 3)
(86, 299, 299, 3)
(86,)


In [0]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [0]:
def build_CNN_classifier():
  
  keep_prob = 0.5

  # 입력 이미지
  x_image = tf.placeholder(tf.float32, [None, 299, 299, 3])

  # 첫번째 convolutional layer - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)한다.
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

  # 첫번째 Pooling layer
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # 두번째 convolutional layer -- 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)한다.
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

  # 두번째 pooling layer.
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # 세번째 convolutional layer
  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

  # 네번째 convolutional layer
  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

  # 다섯번째 convolutional layer
  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)

  # Fully Connected Layer 1 -- 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 된다.
  # 이를 384개의 특징들로 맵핑(maping)한다.
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

  h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

  # Dropout - 모델의 복잡도를 컨트롤한다. 특징들의 co-adaptation을 방지한다.
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

  # 384개의 특징들(feature)을 3개의 클래스로 맵핑(maping)한다.
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 3], stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[3]))
  logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)

  return x_image, y_pred, logits

In [18]:
# Convolutional Neural Networks(CNN) 그래프를 생성합니다.
x_image, y_pred, logits = build_CNN_classifier()

# Cross Entropy를 비용함수(loss function)으로 정의하고, RMSPropOptimizer를 이용해서 비용 함수를 최소화합니다.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

# 정확도를 계산하는 연산을 추가합니다.
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_train, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [19]:
with tf.Session() as sess:

  sess.run(tf.global_variables_initializer())
  sess.run(y_pred, feed_dict={x_image: X_train.astype('float32')})
  print('before loop')
  
  # 10000 Step
  for i in range(10000):
    batch = next_batch(128, X_train, y_train)
    print(True)

    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={X: batch[0], y: batch[1], keep_prob: 1.0})
      loss_print = loss.eval(feed_dict={X: batch[0], y: batch[1], keep_prob: 1.0})

      print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f, 손실 함수(loss): %f" % (i, train_accuracy, loss_print))
      print("Current time is : ", datetime.datetime.now())
    
    sess.run(train_step, feed_dict={X: batch[0], y: batch[1], keep_prob: 0.8})

  test_batch = next_batch(10000, x_test, y_test())
  print("테스트 데이터 정확도: %f" % accuracy.eval(feed_dict={X : test_batch[0], y: test_batch[1], keep_prob: 1.0}))
  

first run


ResourceExhaustedError: ignored